# `importation_processing`
`epidemic-intelligence` is designed to handle large numbers of simulations, but in order to visualize importation data, we must only have one 'run' worth of data. To facilitate this, the `importation_processing` module offers two tools for simplifying and aggregating importation data to make it ready for visualization. 

## summarize_runs
The `importation_processing.summarize_runs` function takes a BigQuery table containg many runs worth of data and reduces it to one run according to some method. 

### Parameters
`client` (`bigquery.Client`): BigQuery client object.

`table_name` (`str`): BigQuery table name containing raw importation data in 'dataset.table' form. 

`destination` (`str`): Name of resulting simplified table in 'dataset.table' form.

`method` (`str`, optional): Method used to determine how data is simplified. Must be one of:
- `'median'` *(default)*: For each time and location, the median value across all runs will be used. A fixed-time metric. 

- `'mean'`: For each time and location, the mean value across all runs will be used. Not recommended. A fixed-time metric. 

- `'mbd'`: Uses modified band depth to compute the most central run, then uses that run's data for all times and locations. A curve-based metric. 

- `'ftvariance'`: Uses the summed variances of each run to compute the most central run, then uses that run's data for all times and locations. A curve-based metric. 

- `'directional'`: Similar to `'ftvariance'`, but first determines whether each `value` is increasing or decreasing at each time step, then uses the summed variances of that data to compute the most central run, then uses that run's data for all times and locations. A curve-based metric. 

`source_column` (`str`, optional): Name of column in original table containing source identifier. Defaults to 'source_basin'.

`target_column` (`str`, optional): Name of column in original table containing target identifier. Defaults to 'target_basin'.

`value_column` (`str`, optional): Name of column in original table containing value identifier, which is what is being tracked. Defaults to 'importations'.

`compartment_column` (`str`, optional): Name of column in original table containing compartment identifier. Defaults to 'compartment'.

---
### Returns
`True` if function executed to completion. 

---

## aggregate_table
The `importation_processing.aggregate_table` function is designed to be used after simplification on a table that only has one run worth of data. It allows you to aggregate by date and compartment to further customize your data for visualization. Note that many parameters carry over from the `simplify_multirun` function, but that using the same `table_name` and `destination` will cause your simplified table to be overwritten. 

### Parameters
`client` (`bigquery.Client`): BigQuery client object.

`table_name` (`str`): BigQuery table name containing importation data in 'dataset.table' form. 

`destination` (`str`): Name of resulting aggregated table in 'dataset.table' form.

`source_column` (`str`, optional): Name of column in original table containing source identifier. Defaults to 'source_basin'.

`target_column` (`str`, optional): Name of column in original table containing target identifier. Defaults to 'target_basin'.

`value_column` (`str`, optional): Name of column in original table containing value identifier, which is what is being tracked. Defaults to 'importations'.

`run_id_column` (`str` or `None`, optional): Name of column in original table containing run ID. If None, then it is assumed all data are part of a single run. Defaults to None.

`compartment_column` (`str`, optional): Name of column in original table containing compartment identifier. Defaults to 'compartment'.

`compartments` (`list` or `bool`, optional): List of compartments to be combined into single compartment in aggregated table. If True, then all compartments will be aggregated. If False, then current compartments will be preserved. Defaults to False. 

`new_compartment` (`str`, optional): If `compartments` is not False, then `new_compartment` will be the name of the aggregated compartment that is created. Defaults to 'compartment'. 

`date` (`str`, optional): The unit of time that data will be aggregated to. Must be one of:
- `'date'` *(default)*: Will preserve current dates. 

- `'epi'`: Will aggregate data into [CDC epiweeks](https://www.cmmcp.org/mosquito-surveillance-data/pages/epi-week-calendars-2008-2024). 'date' column of table must be of type datetime. Will create a temporary table in your BigQuery project, which is deleted upon completion. 

- `'iso'`: Will aggregate data into [ISO weeks](https://en.wikipedia.org/wiki/ISO_week_date). 'date' column of table must be of type datetime.
---
### Returns
`True` if function executed to completion. 

---